## This notebook is almost completely derived from Susan Li's https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8 ##

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from IPython.display import display, HTML
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

## The Data

~~The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe (1/0) a term deposit (variable y).~~

~~This dataset provides the customer information. It includes 41188 records and 21 fields.~~

We'll use the NSCH_2016_topical dataset instead

In [ ]:
fullDF = pd.read_csv('/home/welling/git/synecoace/data/nsch_2016_topical.csv')
print fullDF.columns
subDF=fullDF[fullDF.FIPSST == 45][['ACE3', 'ACE4', 'ACE5', 'ACE6', 'ACE7',
                                   'ACE8', 'ACE9', 'ACE10',
                                   'FWC', 'YEAR', 'FPL', 'SC_AGE_YEARS',
                                   'K4Q30_R', 'K4Q32X01',
                                   'K7Q30', 'K7Q31', 'AGEPOS4']]

data = subDF

print(data.shape)

#data = pd.read_csv('banking.csv', header=0)
data = data.dropna()
print(data.shape)
print(list(data.columns))

In [ ]:
data.head()

## Input variables ##
* SC_AGE_YEARS: selected child age (numeric) -> 'AGE'
* K4Q30_R: Dental care 1=YES 2=YES,other(not dentist) 3=NO -> 'DENTALCARE'
* K4Q32X01: vision tested by eye doctor (2.0 -> False) -> 'VISIONCARE'
* K7Q30: sports teams 1=YES 2=NO -> 'SPORTSTEAMS'
* K7Q31: clubs or organizations 1=YES 2=NO -> 'CLUBS'
* FPL: percent of federal poverty level? (numeric, open-ended)
* AGEPOS4: birth order (numeric) -> 'BIRTHORDER'
* FWC: sample weight
* ~~YEAR: survey year~~ drop this one since 2016 seems to be the only value for these samples

## Predict Variable ##
* ACE3: Parent Divorced -> PARENTDIVORCED
* ACE4: Parent or guardian died -> PARENTDIED
* ACE5: Parent or guardian spent time in jail -> PARENTJAIL
* ACE6: Adult slap/kick/punch others -> SEEPUNCH
* ACE7: experienced violence 1=YES 2=NO -> VIOLENCE
* ACE8: family member mentally ill -> MENTALILL
* ACE9: drugs and alcohol 1=YES 2=NO -> DRUGSALCOHOL
* ACE10: treated unfairly because of race -> RACISM


In [ ]:
data['YEAR'].unique()

In [ ]:
data = data.drop(columns=['YEAR'])
data.head()

In [ ]:
data = data.rename(columns={'AGEPOS4':'BIRTHORDER', 'SC_AGE_YEARS':'AGE', 'K4Q30_R':'DENTALCARE',
                           'K4Q32X01':'VISIONCARE', 'K7Q30':'SPORTSTEAMS', 'K7Q31':'CLUBS',
                            'ACE3':'PARENTDIVORCED',
                            'ACE4':'PARENTDIED', 'ACE5':'PARENTJAIL', 'ACE6':'SEEPUNCH',
                           'ACE7':'VIOLENCE', 'ACE8': 'MENTALILL', 'ACE9':'DRUGSALCOHOL',
                           'ACE10':'RACISM'})
acesL = ['PARENTDIVORCED', 'PARENTDIED', 'PARENTJAIL', 'SEEPUNCH', 'VIOLENCE', 'MENTALILL',
         'DRUGSALCOHOL', 'RACISM']
acesL.sort()
boolColL = ['DENTALCARE', 'VISIONCARE', 'SPORTSTEAMS', 'CLUBS']
boolColL.sort()
scalarColL = ['FPL', 'BIRTHORDER', 'AGE']
scalarColL.sort()

data.head()


In [ ]:
data['AGE'].unique()

In [ ]:
data['BIRTHORDER'].unique()

In [ ]:
out = pd.cut(data['FPL'], 8, labels=False)
data['FPL_quantized'] = out
data['FPL_quantized'].unique()


In [ ]:
massaged_data = data[['FWC', 'AGE', 'FPL_quantized']].copy().rename(columns={'FPL_quantized':'FPL'})
massaged_data['DENTALCARE'] = (data['DENTALCARE'] != 3.0)
massaged_data['VISIONCARE'] = (data['VISIONCARE'] == 1.0)
massaged_data['SPORTSTEAMS'] = (data['SPORTSTEAMS'] == 1.0)
massaged_data['CLUBS'] = (data['CLUBS'] == 1.0)
massaged_data['BIRTHORDER'] = data['BIRTHORDER']
for ace in acesL:
    massaged_data[ace] = (data[ace] == 1)
massaged_data.head()

### Data exploration

In [ ]:
print 'What weighted fraction of samples have each ACE?'
for col in acesL:
    display(massaged_data[[col,'FWC']].groupby([col]).sum()/massaged_data['FWC'].sum())
#massaged_data['VIOLENCE'].value_counts()

Our classes are imbalanced, with many more subjects without ACEs than with.

In [ ]:
def wtThisRow(row, colNm):
    newRow = (row * row['FWC']).drop(['FWC', colNm])
    newRow[colNm] = row[colNm]
    newRow['FWC'] = row['FWC']
    return newRow

def unwtThisRow(row):
    newRow = row / row['FWC']
    return newRow

def weightedMean(df, col):
    meanDF = df.apply(wtThisRow, axis=1, colNm=col).groupby(col).mean()
    return meanDF.apply(unwtThisRow, axis=1).drop('FWC', axis=1)

In [ ]:
print 'Weighted means by ACE status'
for col in acesL:
    display(weightedMean(massaged_data, col))

Observations:

Higher age is associated with higher ACEs, as expected. Higher income is associated with fewer ACEs.

We can calculate categorical means for other categorical variables such as education and marital status to get a more detailed sense of our data.

In [ ]:
for col in scalarColL + boolColL:
    display(weightedMean(massaged_data, col))

Visualizations

In [ ]:
%matplotlib inline
for col in scalarColL:
    for ace in acesL:
        pd.crosstab(massaged_data[col],massaged_data[ace], values=massaged_data.FWC, aggfunc='sum').plot(kind='bar')
        plt.xlabel(col)
        plt.ylabel(ace)
        plt.title('%s by %s' % (ace, col))
        plt.show()

In [ ]:
for col in boolColL:
    for ace in acesL:
        table=pd.crosstab(massaged_data[col],massaged_data[ace], values=massaged_data.FWC, aggfunc='sum')
        table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
        plt.xlabel(col)
        plt.ylabel(ace)
        plt.title('%s fractions by presence of %s' % (ace, col))
        plt.show()

In [ ]:
for col in scalarColL:
    massaged_data[col].hist(weights=data.FWC)
    plt.title('Histogram of %s' % col)
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
def mkSamps(df, nSamp):
    fracWt = df['FWC']/df['FWC'].sum()
    choices = np.random.choice(len(df), nSamp, p=fracWt)
    return df.iloc[choices].drop(columns=['FWC'])


### Create dummy variables

In [ ]:
data = massaged_data.copy()
cat_vars = scalarColL
for var in cat_vars:
    cat_list = pd.get_dummies(data[var], prefix=var)
    data1=data.join(cat_list)
    data=data1

#display(data.head())
    
cat_vars = scalarColL
#print 'cat_vars: ', cat_vars
data_vars = data.columns.values.tolist()
#print 'data_vars: ', data_vars
to_keep=[i for i in data_vars if i not in cat_vars]
#print 'to_keep: ', to_keep
data_final=data[to_keep]
data_final.columns.values

### Deal with weights by creating a new dataset by weighted sampling ###

In [ ]:
data_resampled = mkSamps(data_final, 100000)
data_resampled.head()

### Over-sampling using SMOTE

In [ ]:
print [(col in acesL) for col in data_resampled.columns]

In [ ]:
X = data_resampled.loc[:, [(col not in acesL) for col in data_resampled.columns]]
X.head()
y = {}
for ace in acesL:
    y[ace] = data_resampled.loc[:, [(col == ace) for col in data_resampled.columns]]
for ace, yy in y.items():
    print 'y[%s]: ' % ace
    display(y[ace].head())

In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)
X_train = {}
X_test = {}
y_train = {}
y_test = {}
os_data_X = {}
os_data_y = {}
for ace in acesL:
    X_train[ace], X_test[ace], y_train[ace], y_test[ace] = train_test_split(X, y[ace], test_size=0.3, random_state=0)
    columns = X_train[ace].columns

    os_data_X[ace],os_data_y[ace]=os.fit_sample(X_train[ace], y_train[ace])
    os_data_X[ace] = pd.DataFrame(data=os_data_X[ace],columns=columns )
    os_data_y[ace]= pd.DataFrame(data=os_data_y[ace],columns=[ace])

    # we can Check the numbers of our data
    print("--------")
    print("Sampling for %s" % ace)
    print("length of oversampled data is ",len(os_data_X[ace]))
    print("Number of no subscription in oversampled data",len(os_data_y[ace][os_data_y[ace][ace]==0]))
    print("Number of subscription",len(os_data_y[ace][os_data_y[ace][ace]==1]))
    print("Proportion of no subscription data in oversampled data is ",(float(len(os_data_y[ace][os_data_y[ace][ace]==0]))
                                                                        /float(len(os_data_X[ace]))))
    print("Proportion of subscription data in oversampled data is ",(float(len(os_data_y[ace][os_data_y[ace][ace]==1]))
                                                                        /float(len(os_data_X[ace]))))

### Recursive feature elimination

In [ ]:
data_resampled_vars=data_resampled.columns.values.tolist()
X=[i for i in data_resampled_vars if i not in acesL]

In [ ]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe = {}
cols_after_rfe = {}
for ace in acesL:
    logreg = LogisticRegression()


    rfe[ace] = RFE(logreg, 20)
    rfe[ace] = rfe[ace].fit(os_data_X[ace], os_data_y[ace].values.ravel())
    print '---------'
    print 'Regressing for %s' % ace
    print '---------'
    print(rfe[ace].support_)
    print(rfe[ace].ranking_)
    cols_after_rfe[ace] = os_data_X[ace].columns[rfe[ace].support_]
    print 'selected columns: ', cols_after_rfe[ace]
    display(os_data_X[ace][cols_after_rfe[ace]].head())

### Implementing the model

#### find and remove columns that are poor regressors ####

In [ ]:
import statsmodels.api as sm

cols_after_logit_fit = {}

for ace in acesL:
    X = os_data_X[ace][cols_after_rfe[ace]]
    y = os_data_y[ace][ace]
    logit_model=sm.Logit(y,X)
    result=logit_model.fit(method='lbfgs')
    print '------------'
    print 'Fitting for %s' % ace
    print '------------'
    print(result.summary2())
    cols_after_logit_fit[ace] = [k for k in cols_after_rfe[ace] if result.pvalues[k] < 0.1]
    print 'retaining columns: ', cols_after_logit_fit[ace]
    

#### Repeat regression with the good columns ####

In [ ]:
for ace in acesL:

    X = os_data_X[ace][cols_after_logit_fit[ace]]
    y = os_data_y[ace][ace]
    logit_model=sm.Logit(y,X)
    result=logit_model.fit(method='lbfgs')
    print '------------'
    print 'Fitting for %s' % ace
    print '------------'
    print(result.summary2())
    

### Logistic Regression Model Fitting

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

for ace in acesL:

    X = os_data_X[ace][cols_after_logit_fit[ace]]
    y = os_data_y[ace][ace]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    logreg = LogisticRegression()
    print '---------'
    print 'fitting %s' % ace
    print '---------'
    display(logreg.fit(X_train, y_train))
    y_pred = logreg.predict(X_test)
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    c_mtx = confusion_matrix(y_test, y_pred)
    print 'confusion matrix: '
    print(c_mtx)
    print 'classification report: '
    print(classification_report(y_test, y_pred))

In [ ]:
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_curve
#logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
#fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#plt.figure()
#plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#plt.plot([0, 1], [0, 1],'r--')
#plt.xlim([0.0, 1.0])
#plt.ylim([0.0, 1.05])
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.title('Receiver operating characteristic')
#plt.legend(loc="lower right")
#plt.savefig('Log_ROC')
#plt.show()